In [1]:
from ics import Calendar
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


plt.style.use("ggplot")


In [2]:
with open("data/calendar.ics", "r", encoding="utf-8") as f:
    calendar = Calendar(f.read())

events_data = []

for event in calendar.events:
    if event.begin and event.end:
        duration = (event.end - event.begin).total_seconds() / 3600
        
        events_data.append({
            "event": event.name,
            "start": event.begin.datetime,
            "end": event.end.datetime,
            "duration": duration
        })

df = pd.DataFrame(events_data)
df.head()


,event,start,end,duration
0,党校结业考试,2017-06-03 18:30:00+08:00,2017-06-03 20:00:00+08:00,1.50
1,值机,2017-02-14 14:10:00+08:00,2017-02-14 15:10:00+08:00,1.00
2,Job Hunting,2026-01-27 21:15:00-08:00,2026-01-27 21:30:00-08:00,0.25
3,In Transit,2026-01-09 13:00:00-08:00,2026-01-09 14:15:00-08:00,1.25
4,Work 👩‍💻,2026-01-06 15:15:00-08:00,2026-01-06 16:30:00-08:00,1.25


In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   event     308 non-null    object 
 1   start     308 non-null    object 
 2   end       308 non-null    object 
 3   duration  308 non-null    float64
dtypes: float64(1), object(3)
memory usage: 9.8+ KB


In [4]:
df["start"] = (
    pd.to_datetime(df["start"], utc=True)
      .dt.tz_convert("America/Los_Angeles")
      .dt.tz_localize(None)
)

df["end"] = (
    pd.to_datetime(df["end"], utc=True)
      .dt.tz_convert("America/Los_Angeles")
      .dt.tz_localize(None)
)


df["week"] = df["start"].dt.to_period("W").apply(lambda r: r.start_time)


In [5]:
df_2026 = df[df["start"].dt.year == 2026].copy()

df_2026.head()


,event,start,end,duration,week
2,Job Hunting,2026-01-27 21:15:00,2026-01-27 21:30:00,0.25,2026-01-26
3,In Transit,2026-01-09 13:00:00,2026-01-09 14:15:00,1.25,2026-01-05
4,Work 👩‍💻,2026-01-06 15:15:00,2026-01-06 16:30:00,1.25,2026-01-05
8,Dinner 🥣,2026-01-07 17:15:00,2026-01-07 18:15:00,1.00,2026-01-05
9,Work👩‍💻,2026-01-05 10:30:00,2026-01-05 14:00:00,3.50,2026-01-05


In [6]:
weekly_hours = df_2026.groupby("week")["duration"].sum()

weekly_hours.tail(10)


week
2026-01-05    47.583333
2026-01-12    59.500000
2026-01-19    66.166667
2026-01-26    63.333333
2026-02-02    25.166667
Name: duration, dtype: float64

In [7]:
def categorize(name):
    name = str(name).lower().strip()
    
    # --- Work ---
    if "job hunting" in name or "interview" in name or "resume" in name:
        return "Job Hunting"
    
    if "work" in name:
        return "Work"

    # --- Study / Language ---
    if "study" in name:
        return "Study"
    
    if "french" in name or "duolingo" in name:
        return "French"

    # --- Meals ---
    if "dinner" in name or "brunch" in name or "lunch" in name:
        return "Meals"

    # --- Exercise ---
    if any(k in name for k in ["exercise", "gym", "yoga", "stretch"]):
        return "Exercise"

    # --- Reservations ---
    if "reservation" in name or "booking" in name:
        return "Reservations"

    # --- Chores ---
    if any(k in name for k in ["grocery", "laundry", "household", "haircut"]):
        return "Chores"

    # --- Entertainment ---
    if any(k in name for k in ["watch", "tv", "movie", "netflix", "transit", "nap"]):
        return "Entertainment"

    return "Other"


df_2026["category"] = df_2026["event"].apply(categorize)

df_2026.head()


,event,start,end,duration,week,category
2,Job Hunting,2026-01-27 21:15:00,2026-01-27 21:30:00,0.25,2026-01-26,Job Hunting
3,In Transit,2026-01-09 13:00:00,2026-01-09 14:15:00,1.25,2026-01-05,Entertainment
4,Work 👩‍💻,2026-01-06 15:15:00,2026-01-06 16:30:00,1.25,2026-01-05,Work
8,Dinner 🥣,2026-01-07 17:15:00,2026-01-07 18:15:00,1.00,2026-01-05,Meals
9,Work👩‍💻,2026-01-05 10:30:00,2026-01-05 14:00:00,3.50,2026-01-05,Work


In [8]:
weekly_category = (
    df_2026.groupby(["week", "category"])["duration"]
    .sum()
    .unstack(fill_value=0)
)

weekly_category.tail()


category,Chores,Entertainment,Exercise,French,Job Hunting,Meals,Study,Work
week,,,,,,,,
2026-01-05,2.000000,7.50,2.666667,4.750000,0.000000,9.000000,0.0,21.666667
2026-01-12,3.916667,1.75,1.750000,7.333333,3.000000,10.500000,0.0,31.250000
2026-01-19,6.250000,11.00,2.500000,3.500000,2.000000,9.500000,0.0,31.416667
2026-01-26,2.500000,8.00,3.500000,8.333333,1.666667,11.916667,4.0,23.416667
2026-02-02,0.500000,0.00,0.000000,4.333333,0.000000,4.666667,0.0,15.666667


In [9]:
macro_map = {
    "Work": "Productive",
    "Job Hunting": "Productive",
    "Study": "Productive",
    "French": "Productive",
    
    "Chores": "Maintenance",
    "Meals": "Maintenance",
    
    "Exercise": "Recovery",
    "Entertainment": "Recovery",
}

df_2026["macro_category"] = df_2026["category"].map(macro_map)


In [17]:
plot_df = (
    weekly_category
        .reset_index()
        .melt(id_vars="week",
              var_name="category",
              value_name="hours")
)
weekly_total = plot_df.groupby("week")["hours"].transform("sum")

plot_df["pct"] = plot_df["hours"] / weekly_total

categories = sorted(plot_df["category"].unique())

palette = px.colors.qualitative.Plotly

color_map = {
    cat: palette[i % len(palette)]
    for i, cat in enumerate(categories)
}

fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.08,
    subplot_titles=[
        "Weekly Time Usage",
        "Weekly Trend",
        "Weekly Time Distribution (%)"
    ]
)
for cat in plot_df["category"].unique():
    sub = plot_df[plot_df["category"] == cat]
    
    fig.add_trace(
        go.Bar(
            x=sub["week"],
            y=sub["hours"],
            name=cat,
            legendgroup=cat,
            marker_color=color_map[cat]
        ),
        row=1, col=1
    )
for cat in plot_df["category"].unique():
    sub = plot_df[plot_df["category"] == cat]
    
    fig.add_trace(
        go.Scatter(
            x=sub["week"],
            y=sub["hours"],
            mode="lines+markers",
            name=cat,
            legendgroup=cat,
            marker_color=color_map[cat],
            showlegend=False
        ),
        row=2, col=1
    )
for cat in plot_df["category"].unique():
    sub = plot_df[plot_df["category"] == cat]
    
    fig.add_trace(
        go.Bar(
            x=sub["week"],
            y=sub["pct"],
            name=cat,
            legendgroup=cat,
            marker_color=color_map[cat],
            showlegend=False
        ),
        row=3, col=1
    )
fig.update_layout(
    height=900,
    barmode="stack",
    title="Weekly Time Dashboard"
)

fig.update_yaxes(title_text="Hours", row=1, col=1)
fig.update_yaxes(title_text="Hours", row=2, col=1)
fig.update_yaxes(title_text="%", tickformat=".0%", row=3, col=1)

fig.show()


In [11]:
# weekly_category.tail(6).plot(kind="bar", stacked=True)

# plt.title("Weekly Time Usage")
# plt.ylabel("Hours")
# plt.xticks(rotation=45)
# plt.show()


In [12]:
# weekly_total = plot_df.groupby("week")["hours"].transform("sum")

# plot_df["pct"] = plot_df["hours"] / weekly_total
# fig = px.bar(
#     plot_df,
#     x="week",
#     y="pct",
#     color="category",
#     title="Weekly Time Distribution (100%)"
# )

# fig.update_layout(barmode="stack")
# fig.update_yaxes(tickformat=".0%")
# fig.update_traces(textposition="inside")

# fig.show()


In [13]:
# fig = px.line(
#     plot_df,
#     x="week",
#     y="hours",
#     color="category",
#     markers=True,
#     title="Weekly Trend by Category"
# )

# fig.show()


In [14]:
# fig = px.bar(
#     plot_df,
#     x="week",
#     y="hours",
#     color="category",
#     title="Weekly Time Usage",
# )

# fig.update_layout(barmode="stack")
# fig.show()
